In [1]:
%load_ext sql

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import peakutils
import locate
import farwave
import time

from scipy import signal

MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%matplotlib notebook

In [4]:
%sql sqlite:///schema/test_low.db

u'Connected: None@schema/test_low.db'

In [5]:
l = 0.3 # Length from mic to centre of array

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(sigs, delays, l):
    """ Given delays matrix, calculate angle using both farwave and solver
    """
    # Estimate "location" of sound source, create array record
    f_theta = np.deg2rad(farwave.calc_angle(delays, l))
    for j in range(len(sigs)):
        theta = 0
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            r, theta = locate.locate(delays[j][lr[0]], delays[j][lr[1]], l, r0=5., theta0=f_theta-ANGLE_OFFSET[j])
            theta += ANGLE_OFFSET[j]
            print 'Solver: r = %f, theta = %f' % (r, np.rad2deg(theta))
            break
    
    return np.rad2deg(theta), np.rad2deg(f_theta)


In [6]:
experiments = %sql select id, comment from experiment

Done.


In [7]:
ang_solver_raw = []
ang_solver_butter = []
ang_far_raw = []
ang_far_butter = []

for i in range(len(experiments)):
    cur_experiment_id = int(experiments[i][0])
    result = %sql select data from mic where experiment_id = :cur_experiment_id
        
    mic_0 = np.array(result[0][0])
    mic_1 = np.array(result[1][0])
    mic_2 = np.array(result[2][0])
    
    sigs = [mic_0, mic_1, mic_2]
    #try:
    sigs_cropped, sigs_butter_cropped, offsets, _, _ = locate.crop_sigs_npeaks(sigs)
    
    # Calculate delays, angle using raw signal (cropped)
    delays_raw = calc_delays(sigs_cropped, offsets)
    solver_raw, far_raw = calc_angles(sigs_cropped, delays_raw, l)

    delays_butter = calc_delays(sigs_butter_cropped, offsets)
    solver_butter, far_butter = calc_angles(sigs_butter_cropped, delays_butter, l)
    #except:
    #    print 'Failure to find peaks: Setting to 0'
    #    solver_raw = far_raw = solver_butter = far_butter = 0
        
    print '%f,%f,%f,%f' % (far_raw, far_butter, solver_raw, solver_butter)
    
    ang_solver_raw.append(solver_raw)
    ang_solver_butter.append(solver_butter)
    ang_far_raw.append(far_raw)
    ang_far_butter.append(far_butter)


Done.
Angles: [-59.78294999992787, -75.26306394167504, -44.498538625719476]
Errors: [1.00000717541323, 1.4077649071675558, 1.4019969250163515]

Angle (CW): -59.815465
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.001314, theta = 59.999995


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/pandas/core/window.py:164: ComplexWarning: Casting complex values to real discards the imaginary part
  values = com._ensure_float64(values)


Angles: [-60.93138060937617, -82.26784206403407, -38.92950393280467]
Errors: [1.0001321377608117, 1.2644140330241627, 1.2854797528328477]

Angle (CW): -60.871443
Using microphone 1 as closest mic - (2 left, 0 right)
Solver: r = -0.361549, theta = 0.000277
59.815465,60.871443,59.999995,0.000277
Done.
Angles: [-59.072566264078155, -78.03652765263584, -40.98902657629512]
Errors: [1.0001310201021414, 1.344861008880262, 1.3247924557870216]

Angle (CW): -59.224983
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.005634, theta = 59.999995
Angles: [-59.12387522528797, -86.20791888447866, -32.74425192599108]
Errors: [1.0001169227377271, 1.2032806040833948, 1.1889287702205729]

Angle (CW): -59.213351
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.005321, theta = 59.999995
59.224983,59.213351,59.999995,59.999995
Done.
Angles: [118.40941496302594, 149.453190305103, 90.546809694897]
Errors: [1.000385458834128, 104.78352855705414, 104.78352855705414]

Angle (C

/home/jtmzheng/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Angles: [-58.7133804019804, -89.98783513631152, -28.541741580197748]
Errors: [1.00025218329936, 1.1548421263797612, 1.138343764705342]

Angle (CW): -58.877899
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.007830, theta = 59.999995
240.850203,58.877899,297.131310,59.999995
Done.
Angles: [-36.12638577035296, -65.21266284940499, -24.171734867961984]
Errors: [1.0935649424065328, 1.7342657500159666, 1.0961039521294629]

Angle (CW): -37.746882
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.170950, theta = 59.999996
Angles: [-39.73803788157662, 17.807903057824433, -18.9966037495162]
Errors: [1.0659627858820762, 1.3497148356062212, 1.0575990971524474]

Angle (CW): -16.637460
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 10.535567, theta = 75.058909
37.746882,16.637460,59.999996,75.058909
Done.


/home/jtmzheng/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


Angles: [-36.50990845021982, 17.972808202909373, -15.737896595984665]
Errors: [1.0903591315571095, 1.3462081421848648, 1.038947486537853]

Angle (CW): -13.977221
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 58.670154, theta = 73.356281
Angles: [-36.6174382435634, 13.54277861636309, -19.14274619380682]
Errors: [1.0894724249813474, 1.4515979856112147, 1.058532032139928]

Angle (CW): -17.235555
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.166553, theta = 59.999996
13.977221,17.235555,73.356281,59.999996
Done.
Angles: [-38.30946233190011, -68.41384672092626, -24.431536992384128]
Errors: [1.076201962864659, 1.6094321530525713, 1.0983505475202258]

Angle (CW): -40.042868
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.151861, theta = 59.999996
Angles: [-33.54053907470468, 0.11447839988077407, -24.383866712506364]
Errors: [1.1170066217765755, 1.993106477582047, 1.097935945364129]

Angle (CW): -23.502650
Using microphone 0 as clos

In [8]:
# Assuming comment is the _actual_ angle
for i in range(len(ang_solver_raw)):
    print '%f,%f,%f,%f,%s' % (ang_far_raw[i], ang_far_butter[i], ang_solver_raw[i], ang_solver_butter[i], experiments[i][1])

59.815465,60.871443,59.999995,0.000277,-55
59.224983,59.213351,59.999995,59.999995,-55
240.850203,58.877899,297.131310,59.999995,-55
37.746882,16.637460,59.999996,75.058909,-25
13.977221,17.235555,73.356281,59.999996,-25
40.042868,23.502650,59.999996,59.999996,-25
354.246893,357.572818,419.999996,419.999997,5
357.599562,358.374861,425.935089,419.999997,5
357.826710,358.371502,419.999996,419.999996,5
338.591485,341.399593,419.999995,419.999996,40
341.681739,340.210644,419.999996,419.999996,40
344.019714,343.321324,419.999996,422.754278,40
297.015827,299.825308,299.999983,299.999983,70
292.828094,296.043126,299.999984,299.999983,70
294.689414,299.135529,299.999983,299.999983,70
